In [1]:
import pandas as pd
import numpy as np

---

# Carga de datos

In [13]:
auctions = pd.read_pickle("../../../../data/tp2/auctions_tp2_formateado.pkl")

In [3]:
events = pd.read_pickle("../../../../data/tp2/events_tp2_formateado.pkl")

In [4]:
targets_competencia = pd.read_pickle("../../../../data/tp2/ref_hashes_target.pkl")

In [5]:
ventanas = []
for x in range(1,8):
    ventanas.append("ventana_{}".format(x))

In [6]:
columns = ["date","device_id","dia"] + ventanas
auctions = auctions[columns]
events = events[columns + ["event_id"]]

In [7]:
#Para distinguir despues del merge 
auctions.rename(columns ={"date": "date_auction"},inplace = True)
events.rename(columns ={"date": "date_event"},inplace = True)

In [8]:
auctions_events = pd.merge_asof(events,auctions, left_on = "date_event", 
                               right_on = "date_auction", by = "device_id",
                               suffixes = ("_event","_auction"))

In [9]:
del events

---

### De auctions- events me quedo con sólo datos que correspondan a la competencia

In [11]:
auctions_events_competencia = auctions_events.loc[auctions_events["device_id"].isin(targets_competencia.index)]

---

# Cargo features generados hasta el momento

## Sets de entrenamiento

In [17]:
# Ventanas de tres dias
Xs = {}
print("Cargando features")
for ventana_nro in range(1, 7):
    try:
        Xs[ventana_nro] = pd.read_csv("../../features/ventana_{}_entrenar_relaciones.csv".format(ventana_nro), index_col=0)
        print("Ventana {} agregada".format(ventana_nro))
    except FileNotFoundError:
        print("No se econtro la ventana {}, se generará".format(ventana_nro))
        Xs[ventana_nro] = auctions.loc[auctions["ventana_{}".format(ventana_nro)]]["device_id"] \
            .drop_duplicates().to_frame().set_index("device_id").copy()

Cargando features
Ventana 1 agregada
Ventana 2 agregada
Ventana 3 agregada
Ventana 4 agregada
Ventana 5 agregada
Ventana 6 agregada


## Set que se usará para predecir

In [18]:
# Features para hacer las predicciones que se suben, corresponden a la ventana 7
Xs_predecir = {}

try:
    print("Cargando features usados para predecir")
    Xs_predecir[7] = pd.read_csv("../../features/predecir_relaciones.csv", index_col=0)
except FileNotFoundError:
    print("No se encontraron los features usados para predecir, se generará el dataframe requerido")
    Xs_predecir[7] = targets_competencia.copy()

Cargando features usados para predecir


In [19]:
features = {}
# Todos los datasets tienen la misma cantidad y nombre de features, tomo la ventana 1
columnas_ya_generadas = list(Xs[6].columns.values)

## Cantidad de eventos luego de la ultima subastas dentro de la ventana

In [23]:
def cantidad_eventos_desde_ultima_subasta(dataframe,nro_ventana): 
    idx = dataframe.groupby(["device_id"])["date_auction"].transform(max) == dataframe["date_auction"]
    dataframe = dataframe[idx]
    
    feature = dataframe.groupby(["device_id"])["date_event"].agg({"date_event":"count"})\
                    .rename(columns = {"date_event":"cantidad_eventos_desde_ultima_subasta_hasta_fin_ventana"})
     
    return feature["cantidad_eventos_desde_ultima_subasta_hasta_fin_ventana"]


In [24]:
features["cantidad_eventos_desde_ultima_subasta_hasta_fin_ventana"] = (cantidad_eventos_desde_ultima_subasta, ["cantidad_eventos_desde_ultima_subasta_hasta_fin_ventana"])

### Genero los features

In [28]:
def generar_feature_en_ventanas(dataframe, generador_feature, destinos, ventana_inicial, ventana_final,nombre_data_merge):
    """El rango se toma como python, [ventana_inicial, ventana_final)"""
    for ventana_nro in range(ventana_inicial, ventana_final):
        print("generando feature en v{}".format(ventana_nro))
       
        condicion = dataframe["ventana_{}_auction".format(ventana_nro)] & dataframe["ventana_{}".format(ventana_nro) +"_"+nombre_data_merge]
        
        feature = generador_feature(dataframe.loc[condicion], ventana_nro)
        destinos[ventana_nro] = destinos[ventana_nro].merge(feature, left_index=True, right_index=True, how="left")
        destinos[ventana_nro].fillna(0,inplace = True)

In [39]:
COLUMNAS_QUE_GENERA_EL_FEATURE = 1
FUNCION_PARA_CALCULAR_EL_FEATURE = 0

for nombre_feature in features:    
    columnas_generadas = features[nombre_feature][COLUMNAS_QUE_GENERA_EL_FEATURE]
    feature_ya_generado = False
    for columna in columnas_generadas:
        if columna in columnas_ya_generadas:
            feature_ya_generado = True
            break
    
    if feature_ya_generado:
        continue
    
    print("Generando feature {} en ventanas".format(nombre_feature))
    generar_feature_en_ventanas(auctions_events, features[nombre_feature][FUNCION_PARA_CALCULAR_EL_FEATURE], Xs, 1, 7,"event")
    print("Generando feature {} en set de predicciones".format(nombre_feature))
    generar_feature_en_ventanas(auctions_events_competencia, features[nombre_feature][FUNCION_PARA_CALCULAR_EL_FEATURE], Xs_predecir, 7, 8, "event")
    columnas_ya_generadas.extend(features[nombre_feature][COLUMNAS_QUE_GENERA_EL_FEATURE])

In [43]:
for ventana_nro in range(1, 7):
    Xs[ventana_nro].to_csv("../../features/ventana_{}_entrenar_relaciones.csv".format(ventana_nro))

Xs_predecir[7].to_csv("../../features/predecir_relaciones.csv")